In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter


In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1.543355,-1.003174,2.56695,1.25707,-0.500615,0.882139,0.871338,1.979825,1.71022,-0.520621
1,-0.239005,0.039737,2.121197,0.443353,0.220981,0.576949,-0.225159,2.280511,3.164198,-1.077476
0,-1.383168,-1.225669,0.686294,-1.506217,1.091092,0.628869,-2.399749,1.015197,0.885706,1.469953
0,0.554796,-0.790668,2.332014,-0.628538,0.023144,0.476495,1.366467,1.095272,-0.722094,0.476656
1,0.743231,-1.149502,0.03855,0.236622,-0.184346,0.437714,-0.660908,0.054314,-0.357087,1.002649


## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Scaling and Imputation
2. Fscore
3. Mutual Information Score
4. MRMR feature selection strategies
5. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Scaling and Imputation

In [4]:
import dsds.transform as t

In [5]:
features = df.columns
features.remove("target")

In [6]:
scaled = t.scale(df, cols=features, strategy="standard")
scaled.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,0.957481,-0.660313,0.937454,0.757682,-0.233091,1.271435,0.885149,0.913611,0.380189,-0.617029
1,-0.149863,0.026282,0.730656,0.358838,0.262408,0.62985,0.180897,1.192673,1.034733,-0.950429
0,-0.860708,-0.806792,0.064962,-0.596745,0.859887,0.739,-1.215785,0.018355,0.009014,0.574766
0,0.343309,-0.520411,0.82846,-0.16655,0.126559,0.418672,1.203157,0.092671,-0.714776,-0.01994
1,0.46038,-0.756648,-0.235546,0.257509,-0.015919,0.337144,-0.098972,-0.873425,-0.55046,0.294983


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# The difference in result is caused by using ddof = 1 for sample variance in dsds
# and using ddof = 0 in sklearn.

# Long and convoluted code just to do some scaling...
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
# scaled2[:5, :] # scaled2 is a numpy matrix
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]
scaled2.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,target
0,0.957485,-0.660317,0.937459,0.757686,-0.233092,1.271441,0.885153,0.913616,0.380191,-0.617032,1
1,-0.149863,0.026283,0.730660,0.358840,0.262409,0.629853,0.180898,1.192679,1.034738,-0.950433,1
2,-0.860712,-0.806796,0.064962,-0.596748,0.859891,0.739004,-1.215791,0.018355,0.009014,0.574769,0
3,0.343311,-0.520414,0.828464,-0.166551,0.126559,0.418674,1.203163,0.092672,-0.714780,-0.019940,0
4,0.460382,-0.756652,-0.235547,0.257510,-0.015919,0.337146,-0.098973,-0.873430,-0.550462,0.294984,1


In [9]:
%%timeit
scaled = t.scale(df, cols=features, strategy="standard")

2.67 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]


15.3 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
import dsds.transform as t
t.impute(df, cols=features, strategy="median").head(3) 

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1.543355,-1.003174,2.56695,1.25707,-0.500615,0.882139,0.871338,1.979825,1.71022,-0.520621
1,-0.239005,0.039737,2.121197,0.443353,0.220981,0.576949,-0.225159,2.280511,3.164198,-1.077476
0,-1.383168,-1.225669,0.686294,-1.506217,1.091092,0.628869,-2.399749,1.015197,0.885706,1.469953


In [12]:
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)
imputed.head(3)

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,1.0,1.543355,-1.003174,2.566950,1.257070,-0.500615,0.882139,0.871338,1.979825,1.710220,-0.520621
1,1.0,-0.239005,0.039737,2.121197,0.443353,0.220981,0.576949,-0.225159,2.280511,3.164198,-1.077476
2,0.0,-1.383168,-1.225669,0.686294,-1.506217,1.091092,0.628869,-2.399749,1.015197,0.885706,1.469953


In [31]:
%%timeit
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)


120 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
t.impute(df, cols=features, strategy="median")

1.19 ms ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Fscore

In [14]:
import dsds.fs as fs # fs = feature_selection
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [15]:
fs._f_score(df, target=target, num_list = features)

array([3.30628455e-01, 3.25840752e-01, 2.50950030e+03, 1.06995769e+04,
       2.32917726e+01, 4.82412614e+02, 1.14142508e+04, 3.19484329e-01,
       2.01306541e+04, 9.58264519e+03])

In [16]:
# The more core, the bigger the difference. Data here is not big enough to show the difference
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.00s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",0.330628,0.565291
"""column_1""",0.325841,0.56812
"""column_2""",2509.500296,0.0
"""column_3""",10699.576897,0.0
"""column_4""",23.291773,0.000001
"""column_5""",482.412614,1.1420e-106
"""column_6""",11414.250763,0.0
"""column_7""",0.319484,0.571919
"""column_8""",20130.654098,0.0


In [17]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


,feature,f_value,p_value
0,column_0,0.330628,5.652906e-01
1,column_1,0.325841,5.681198e-01
2,column_2,2509.500296,0.000000e+00
3,column_3,10699.576897,0.000000e+00
4,column_4,23.291773,1.393990e-06
5,column_5,482.412614,1.142039e-106
6,column_6,11414.250763,0.000000e+00
7,column_7,0.319484,5.719190e-01
8,column_8,20130.654098,0.000000e+00
9,column_9,9582.645191,0.000000e+00


In [18]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.01s in computing Fscore.


,feature,f_value,p_value
0,column_0,0.330628,5.652906e-01
1,column_1,0.325841,5.681198e-01
2,column_2,2509.500296,0.000000e+00
3,column_3,10699.576897,0.000000e+00
4,column_4,23.291773,1.393990e-06
5,column_5,482.412614,1.142039e-106
6,column_6,11414.250763,0.000000e+00
7,column_7,0.319484,5.719190e-01
8,column_8,20130.654098,0.000000e+00
9,column_9,9582.645191,0.000000e+00


# Mutual Information Score

In [19]:
# Vastly faster than sklearn. Finished in 0.7s in this run
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 10/10 [00:00<00:00, 12.84it/s]


feature,estimated_mi
str,f64
"""column_4""",0.165045
"""column_8""",0.110466
"""column_3""",0.110329
"""column_0""",0.063793
"""column_6""",0.056662
"""column_9""",0.049541
"""column_2""",0.024717
"""column_5""",0.013657
"""column_7""",0.010428


In [20]:
# A wrapper for more apples to apples comparison
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [21]:
# The reason sklearn's impl is slow is that it did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality. Finished in 4.4s in this run
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_4""",0.165051
"""column_8""",0.110466
"""column_3""",0.110329
"""column_0""",0.063793
"""column_6""",0.056662
"""column_9""",0.049541
"""column_2""",0.024717
"""column_5""",0.013658
"""column_7""",0.010428


# MRMR Feature selection Strategy

In [22]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [23]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [24]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 10/10 [00:05<00:00,  1.67it/s]

Spent 8.75s to compute mrmr.


['column_8',
 'column_6',
 'column_3',
 'column_9',
 'column_2',
 'column_5',
 'column_4',
 'column_7',
 'column_0',
 'column_1']

In [25]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 2226.63it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.01s in computing.


['column_8',
 'column_6',
 'column_3',
 'column_9',
 'column_2',
 'column_5',
 'column_4',
 'column_7',
 'column_0',
 'column_1']

In [26]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 588.17it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.02s in computing.


['column_8',
 'column_6',
 'column_3',
 'column_9',
 'column_2',
 'column_5',
 'column_4',
 'column_7',
 'column_0',
 'column_1']

# Power Transform

In [27]:
start = perf_counter()
res_eager = t.power_transform(df, cols=features, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Spent 0.20s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1.528213,-1.11014,2.866121,1.250044,-0.490738,1.012572,0.851723,2.011809,1.584071,-0.509153
1,-0.239492,0.039536,2.337013,0.442306,0.223097,0.635259,-0.226727,2.321146,2.818786,-1.034578
0,-1.395827,-1.380078,0.714706,-1.515932,1.133764,0.697603,-2.519505,1.025126,0.845522,1.548555
0,0.552393,-0.859452,2.586001,-0.630555,0.023168,0.516899,1.322975,1.106648,-0.751349,0.486649
1,0.739109,-1.286846,0.038654,0.236306,-0.182875,0.472023,-0.673053,0.05435,-0.364832,1.042299


In [28]:
from sklearn.preprocessing import power_transform

In [29]:
# Sklearn with Pandas

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 0.69s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,1,1.528213,-1.110140,2.866121,1.250044,-0.490738,1.012572,0.851723,2.011809,1.584071,-0.509153
1,1,-0.239492,0.039536,2.337013,0.442306,0.223097,0.635259,-0.226727,2.321146,2.818786,-1.034578
2,0,-1.395827,-1.380078,0.714706,-1.515932,1.133764,0.697603,-2.519505,1.025126,0.845522,1.548555
3,0,0.552393,-0.859452,2.586001,-0.630555,0.023168,0.516899,1.322975,1.106648,-0.751349,0.486649
4,1,0.739109,-1.286846,0.038654,0.236306,-0.182875,0.472023,-0.673053,0.054350,-0.364832,1.042299


# GC

In [30]:
import gc 
gc.collect()

115